# Adding Migration to Location Selection

In [1]:
import tqdm
import pickle
import numpy as np
import pandas as pd

import rasterio
from rasterio.transform import Affine
from rasterio.windows import from_bounds

from scipy.stats import norm as scipy_norm

In [2]:
spnames = ["actitis_hypoleucos", "spinus_spinus", "turdus_merula", "turdus_pilaris"]
p = len(spnames)

In [3]:
#
# This is only required to get lat_grid and height
#
min_lon, max_lon = 21, 30
min_lat, max_lat = 59.7, 62.7
map_path = "/hpc/group/dunsonlab/szw3/finland/data/location_maps/"+spnames[0]+"_a.tif"
with rasterio.open(map_path) as src:
    window = from_bounds(min_lon, min_lat, max_lon, max_lat, transform=src.transform)
    a = src.read(1, window=window)
    transform = src.window_transform(window)
    height, width = a.shape[0], a.shape[1]

ys = np.arange(height)
_, lat_grid = rasterio.transform.xy(transform, ys, np.zeros_like(ys))
lat_grid = np.array(lat_grid)

lat_grid

array([62.69955651, 62.69866953, 62.69778256, ..., 59.70245967,
       59.70157269, 59.70068572])

In [4]:
m_path = "/hpc/group/dunsonlab/szw3/finland/data/migration_prior_params.pickle"
with open(m_path, "rb") as handle:
    m_params = pickle.load(handle)
index_style = dict(zip(m_params.index, [x.lower().replace(" ", "_") for x in m_params.index]))
m_params.rename(index=index_style, inplace=True)

m_params.head()

,co.first.1,co.first.2,co.last.1,co.last.2,pm.first,pm.last,day1,day2
chloris_chloris,-20.000000,0.000000,400.000000,0.000000,1.000000,1.000000,0.0,0.0
ficedula_hypoleuca,-9.585293,2.374318,192.044640,0.061587,25.267569,27.779167,0.0,0.0
turdus_iliacus,-258.374546,5.823682,775.810244,-8.392168,28.300215,70.618044,0.0,0.0
phylloscopus_trochilus,-56.348579,2.960777,588.817703,-6.016158,14.065667,48.763756,0.0,0.0
turdus_merula,-807.191935,13.315181,1135.277672,-12.671696,67.102577,94.550687,0.0,0.0


In [5]:
#
# can load this directly from model_utils instead
#
def m_numpy(lats, days, theta):
    Z1 = (days - (theta[0] + theta[1]*lats))/(theta[4]/2)
    Z2 = (days - (theta[2] + theta[3]*lats))/(theta[5]/2)
    P1 = scipy_norm.cdf(Z1)
    P2 = 1-scipy_norm.cdf(Z2)
    return np.c_[P1, P2].min(axis=1)

# New code below

The chunk below computes a [H, p] matrix of migration probabilities on a given day t. This should be run inside our main loop. We can prevent species which have not arrived from influencing the utility by zeroing out the corresponding entries of \delta.

In [6]:
# starting date (May 1)
jday_0 = 121

# day of experiment
t = 0

m_mat = np.zeros([height, p])
jday_t = jday_0 + t
for j in tqdm.tqdm(range(p)):
    theta_j = m_params.loc[spnames[j]][:6].to_numpy()
    m_mat[:,j] = m_numpy(lat_grid, jday_t, theta_j)
m_mask = m_mat > 0.5

100%|██████████| 4/4 [00:00<00:00, 714.05it/s]


In [7]:
#
# integration with the main loop below
#
for t in range(T):    
    m_mat = np.zeros([height, p])
    jday_t = jday_0 + t
    for j in range(p):
        theta_j = m_params.loc[spnames[j]][:6].to_numpy()
        m_mat[:,j] = m_numpy(lat_grid, jday_t, theta_j)
    m_mask = m_mat > 0.5
    delta_filtered = delta*m_mask[:,None,:]
    
    if t == 0:
        U_active = np.sum(delta_filtered, axis=2)
    else:
        #
        # ...
        #
        U_active = np.sum(delta_filtered*repulsion, axis=2)

NameError: name 'T' is not defined

# Code for randomly choosing locations

In [8]:
valid_indices = np.argwhere(U_local > 0)
row, col = valid_indices[np.random.choice(len(valid_indices))]

NameError: name 'U_local' is not defined